In [2]:
import pandas as pd
import numpy as np 
import glob
from biopandas.pdb import PandasPdb
import warnings
warnings.filterwarnings("ignore")
from get_distances import *
import sys
from multiprocessing import Pool
import time
import math
import datetime
import pickle
import os

In [3]:
# example filenames
# output_dir + os.path.basename("/rcfs/projects/proteometer/human_proteome_precaculated/TEST_stability_precalculated.csv")[:-4] +  "_output.csv"

In [4]:
# read in stability data
full_stability = pd.read_csv("/rcfs/projects/proteometer/human_proteome_precaculated/stability_precaculated.csv")
full_stability


,Unnamed: 0,protein_acc,wild_type,position,plddt,foldx_ddg_min,foldx_ddg_max,foldx_ddg_abs_max,foldx_ddg_abs_median
0,0,A0A0C4DH31,M,1,49.74,-0.116889,0.374417,0.374417,0.058460
1,1,A0A0C4DH31,D,2,48.50,-0.255073,3.169780,3.169780,0.267710
2,2,A0A0C4DH31,W,3,55.08,-0.182149,2.033680,2.033680,0.210595
3,3,A0A0C4DH31,T,4,62.37,-0.275341,0.477429,0.477429,0.236620
4,4,A0A0C4DH31,W,5,58.32,-0.412867,1.197650,1.197650,0.497052
...,...,...,...,...,...,...,...,...,...
8544118,2376,Q9Y490,Q,2529,49.67,-0.574934,3.128290,3.128290,0.346243
8544119,2377,Q9Y490,Y,2530,42.18,-0.407749,2.709540,2.709540,1.348430
8544120,2378,Q9Y490,K,2531,44.64,-0.426433,1.239740,1.239740,0.266512
8544121,2379,Q9Y490,F,2532,43.24,-0.630152,5.610040,5.610040,0.376770


In [5]:
unique_uniprots = full_stability['protein_acc'].unique()
len(unique_uniprots)
# 20437 / 50 = 408.74

20437

In [6]:
# from https://stackoverflow.com/questions/312443/how-do-i-split-a-list-into-equally-sized-chunks
def split_list(lst, n):
    """Splits a list into n equal parts."""
    k, m = divmod(len(lst), n)
    return [lst[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n)]


In [9]:
list_of_chunks = split_list(unique_uniprots, 50)
list_of_chunks

[array(['A0A0C4DH31', 'A0A075B6S0', 'A0A075B767', 'P24666', 'O43927',
        'Q9H1Z8', 'Q9UHV8', 'Q9BW30', 'Q7Z5A8', 'P12273', 'Q8IWF7',
        'P20962', 'Q6UWQ7', 'P12872', 'Q8N9A8', 'Q99731', 'Q9H7B7',
        'A0A599', 'Q9BYR5', 'Q9H2X8', 'Q7Z7G2', 'P04921', 'Q15369',
        'Q8N6M8', 'Q8WW36', 'Q9UGC6', 'Q8WTT0', 'P58549', 'Q8N5I9',
        'P63313', 'P0DPF5', 'Q8TCD6', 'Q5GAN6', 'A0A494BZU4', 'Q9Y2Q3',
        'P0DPP9', 'Q6NUI1', 'P11684', 'Q9Y5J9', 'P27482', 'Q9BSU3',
        'P60033', 'P25713', 'Q9P086', 'Q8WWF3', 'P02794', 'P84098',
        'P02741', 'Q16653', 'P0CG04', 'Q9Y3E5', 'Q96D03', 'P25815',
        'O60936', 'O60543', 'Q9Y3D9', 'Q5TGI4', 'Q6ZUF6', 'Q8IZJ0',
        'O15522', 'P26715', 'Q7Z6I8', 'Q96A26', 'Q6ZNB5', 'A0A075B706',
        'A0A0B4J2A2', 'Q7Z5A9', 'Q9H2J1', 'P0C0S5', 'A0A5A2', 'P14174',
        'Q8N9T2', 'Q9BYR6', 'A0A0C4DH32', 'Q16661', 'Q92567', 'Q6UWF9',
        'P51397', 'B2RPK0', 'P13501', 'A0A6Q8PGS0', 'Q9NZ81', 'O43812',
        'Q9NRR1', 'O75347'

In [13]:
output_dir = "/qfs/projects/proteometer/pheno_analysis/split_stability_input"
counter = 1
for uniprot_chunk in list_of_chunks:
    print(counter)
    chunk_of_stability = full_stability[full_stability['protein_acc'].isin(uniprot_chunk)]
    # save csv
    chunk_of_stability.to_csv(output_dir + f"/stablity_chunk_{str(counter)}.csv")
    counter = counter + 1
    



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
